It is possible to use this package to build custom circuits out of matchgates. To do this, follow the instructions below

In [28]:
from matchgates import Circuit, MatchGate, AppliedMatchGate
import numpy as np
import scipy.linalg as la

First, build the desired MatchGate objects. There are three choices of constructor for a MatchGate:

(a) Specify the coefficients of each Pauli generator in a dictionary.

(b) Provide the Unitary matrix representation of a matchgate directly as an np.ndarray.

(c) Provide two matrices A & B with det(A) = det(B).

As a simple example consider the matchgate:  

$U = e^{-iXX}$

In [29]:
param_dict = MatchGate.empty_param_dict()  # empty dictionary of parameters for all possible Pauli generators (z1, z2, zz, xx, xy, yx, yy)
param_dict["xx"] = -1.0
U = MatchGate(param_dict)
print(U.param_dict["xx"])

# To construct the matchgate from a Unitary, use the from_unitary class method.
X = np.array([[0, 1], [1, 0]])
XX = np.kron(X, X)
unitary = la.expm(-1j * XX)
U = MatchGate.from_unitary(unitary)
print(
    U.param_dict["xx"]
)  # check that it gives correct value of parameters for XX generator.

# To construct the matchgate from A & B matrices, we use the from_AB class method.
A = np.ndarray((2, 2), dtype=np.complex128)
B = np.ndarray((2, 2), dtype=complex)
A[0, 0] = unitary[0, 0]
A[1, 1] = unitary[3, 3]
A[0, 1] = unitary[0, 3]
A[1, 0] = unitary[3, 0]
B[0, 0] = unitary[1, 1]
B[1, 0] = unitary[2, 1]
B[0, 1] = unitary[1, 2]
B[1, 1] = unitary[2, 2]

U = MatchGate.from_AB(A, B)
print(U.param_dict["xx"])

-1.0
-1.0
-1.0


To create a circuit from this matchgate, we need to specify which qubits it acts on. Since matchgate circuits are only defined on nearest neighbour connectivity, the qubits are labelled from 0 to N. We can collect this information into an AppliedMatchGate object that can then be fed into a circuit:

In [30]:
N = 10  # number of qubits in system.
gate1 = AppliedMatchGate(U, N, [0, 1])  # AppliedMatchGate objects take the arguments:
gate2 = AppliedMatchGate(
    U, N, [7, 8]
)  # (MatchGate, number of qubits of whole system, which pair of qubits the gate acts on)

circuit = Circuit(
    N, [gate1, gate2]
)  # Circuits just need the number of qubits of the whole system and a list of AppliedMatchGate objects.

Now it is possible to run this circuit in the same way that the Ising and XY circuits are run in the example notebook: `examples/noiseless_circuits.ipynb`